In [1]:
import mxnet as mx
from mxnet import nd, autograd as ag, gluon
from get_data import get_data

In [2]:
# Data

num_input = 2
num_output = 1
num_examples = 1000
batch_size = 10

train_data,train_label = get_data(W=[3,-6],b=9,num_data=num_examples)
X = nd.array(train_data)
Y = nd.array(train_label)

batch_size = 10
train_data = gluon.data.DataLoader(
    gluon.data.ArrayDataset(X,Y),
    batch_size=batch_size,
    shuffle=True)

In [3]:
# Model

net = gluon.nn.Dense(1, in_units=2) # input 2维，output 1维
net.collect_params().initialize(mx.init.Normal(sigma=1.))

squire_loss = gluon.loss.L2Loss()

In [4]:
# Train

trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':.01})

epochs = 5
model_ctx = mx.cpu()

for e in range(epochs):
    cumulative_loss = 0
    for i,(data,label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        with ag.record():
            output = net(data)
            loss = squire_loss(output,label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.mean(loss).asscalar()
    print("%s: %s"%(e,cumulative_loss/num_examples))

print(net.weight.data(), net.bias.data())

0: 2.77466153049
1: 0.374278589487
2: 0.0510698633268
3: 0.0073281574389
4: 0.00141206835187

[[ 2.99173284 -5.9478898 ]]
<NDArray 1x2 @cpu(0)> 
[ 8.94240856]
<NDArray 1 @cpu(0)>
